In [ ]:
# test (miniconda3 (Python 3.12.2), open integrated terminal)

In [1]:
import csv

In [2]:
# format prices into lists
close_prices = []
data = []
open_prices = []

with open('./data/prices/csvs/AAPL.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        data.append(float(row[1]))

data.reverse()
print(data)

# for i in range(0,10):
#     if close_prices[i+5] > close_prices[i+4]:
#         m = 'up'
#     else:
#         m = 'down'
#     print(close_prices[i:i+5], close_prices[i+5], m)


[39.48, 35.55, 37.07, 36.98, 37.69, 38.33, 38.45, 38.07, 37.5, 38.27, 38.74, 38.97, 39.21, 38.33, 38.48, 38.18, 39.44, 39.08, 38.67, 41.31, 41.61, 41.63, 42.81, 43.55, 43.56, 42.74, 42.6, 42.36, 42.72, 42.55, 42.7, 42.61, 42.73, 43.01, 42.77, 43.24, 43.56, 43.58, 43.72, 43.29, 43.74, 43.96, 43.88, 43.63, 43.13, 43.23, 44.73, 45.23, 45.43, 45.93, 46.53, 47.01, 46.63, 47.04, 48.77, 47.76, 47.19, 46.7, 47.12, 47.18, 47.49, 47.81, 48.51, 48.84, 48.92, 49.25, 50.03, 49.88, 50.16, 49.74, 49.72, 49.81, 49.81, 50.78, 50.97, 51.13, 51.87, 51.79, 51.32, 51.08, 51.15, 50.17, 52.63, 52.29, 52.94, 52.12, 50.72, 50.73, 50.18, 49.3, 46.43, 47.17, 47.73, 47.52, 47.25, 45.77, 46.65, 45.7, 44.92, 44.74, 44.56, 44.35, 44.58, 43.77, 43.33, 44.91, 45.64, 46.31, 47.54, 48.15, 48.7, 48.55, 48.54, 48.19, 48.47, 49.61, 49.47, 49.87, 49.7, 49.65, 48.89, 49.95, 49.94, 49.48, 50.39, 50.68, 51.1, 51.06, 50.01, 50.31, 50.81, 50.44, 50.83, 51.3, 51.13, 50.84, 51.42, 50.65, 51.81, 52.21, 52.17, 51.76, 51.94, 52.42, 5

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data, columns=['Close Price'])

df['Date'] = pd.date_range(start='2023-07-01', periods=len(data), freq='D')

df['Moving Average'] = df['Close Price'].rolling(window=21).mean()

plt.figure(figsize=(10, 5))
plt.plot(df['Date'], df['Close Price'], label='Close Price', marker='.', markersize=4, linewidth=1)
plt.plot(df['Date'], df['Moving Average'], label='21-Day Moving Average', linestyle='--', linewidth=1)

plt.title('Close Price and Moving Average')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

plt.grid(True)
plt.show()


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
from langchain_openai import ChatOpenAI
import openai
import getpass
import os
import langchainhub
from langchain import hub

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

completion = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "user",
            "content": "2+2="
            # "content": "these are the stock prices (closing price) from past 5 days: [37.07, 36.98, 37.69, 38.33, 38.45], last element of the list is the latest closing price. these are the news headlines relating to the stock's company: ['may have reached its all time high', 'is the stock price going to fall?']. predict if the stock price next day will go up or down. your answer should either be up or down, do not elaborate.",
            # "content": "these are the stock prices (closing price) from past 5 days: [37.07, 36.98, 37.69, 38.33, 38.45], last element of the list is the latest closing price. predict if the stock price next day will go up or down. your answer should either be up or down, do not elaborate.", # here i could iteratively pass stock prices?
        },
    ],
)
print(completion.choices[0].message.content)


4


In [ ]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages
print(example_messages[0].content)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("When did Katharine Braddick join Barclays?"):
    print(chunk, end="", flush=True)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")